<font size="5"><b>Mining Healthcare Data to Predict Maternal Mortality Rates in Cameroon Using Classification</b></font>

<font size="4">

**Objective:** Build a data warehouse for healthcare records and use classification techniques to predict maternal mortality risks.

**Deliverables:**
- Data Warehouse Design
- ETL Process
- OLAP Queries
- Classification Algorithm: Use Naive Bayes or Random Forest to predict risks.
- Insights into improving maternal healthcare services.

</font>

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import os

# !pip install sqlalchemy pymysql pandas matplotlib seaborn scikit-learn folium


### Database Connection Configuration

The code snippet below defines a database connection configuration using the following parameters:

- **Host:** gateway01.eu-central-1.prod.aws.tidbcloud.com
- **Port:** 4000
- **User:** 3e6VMfsfv7ahrg9.root
- **Password:** M3cQSRGCTM6SNXGK
- **Database:** test
- **SSL CA:** The SSL certificate file path is set to the absolute path of 'warehouse/isrgrootx1.pem'

The code then creates a connection URL for SQLAlchemy using the provided database configuration parameters. It includes the user, password, host, port, database, and SSL CA file path in the connection URL.

Finally, the code establishes a database connection using the SQLAlchemy `create_engine` function with the connection URL.

In [2]:
# Database connection configuration
db_config = {
    'host': "gateway01.eu-central-1.prod.aws.tidbcloud.com",
    'port': 4000,
    'user': "3e6VMfsfv7ahrg9.root",
    'password': "M3cQSRGCTM6SNXGK",
    'database': "test",
    'ssl_ca': os.path.abspath('Data-Mining/data/permission/isrgrootx1.pem')
}

# Create connection URL for SQLAlchemy
connection_url = (
    f"mysql+pymysql://{db_config['user']}:{db_config['password']}"
    f"@{db_config['host']}:{db_config['port']}/{db_config['database']}?ssl_ca={db_config['ssl_ca']}"
)

# Establish the database connection
engine = create_engine(connection_url)

### Table Creation Queries

The code snippet contains a series of SQL queries to create various dimension and fact tables for a healthcare data warehouse. Here are the tables being created:

- **DIM_DATE:** Contains attributes related to dates such as year, month, and quarter.
- **DIM_HEALTH_INITIATIVE:** Describes health initiatives with start and end dates.
- **DIM_HEALTHCARE_FACILITY:** Includes details about healthcare facilities like type, capacity, and location.
- **DIM_INFRASTRUCTURE:** Focuses on infrastructure information like road quality and access to utilities.
- **DIM_LOCATION:** Captures location-specific data such as population density and regional policies.
- **DIM_MEDICAL_HISTORY:** Records historical medical information of patients.
- **DIM_PATIENT:** Stores patient details like age, education level, and antenatal visits.
- **DIM_PATIENT_OUTCOME:** Tracks patient outcomes related to childbirth and mortality.
- **DIM_SOCIOECONOMIC:** Covers socioeconomic factors like income level and access to healthcare.
- **DIM_HEALTHWORKER:** Contains data about healthcare workers including experience and specialization.
- **FACT_MATERNAL_MORTALITY:** Represents the fact table for maternal mortality data, linking various dimension keys with information about births and maternal deaths.

Each table creation query ensures that the tables are created if they do not already exist, following the specified schema for each dimension and fact table.

In [3]:
# Table creation queries
tables = {
    "DIM_DATE": """
        CREATE TABLE IF NOT EXISTS DIM_DATE (
            date_key INT PRIMARY KEY,
            full_date DATE,
            year INT,
            month INT,
            month_name VARCHAR(20),
            week_number INT,
            quarter INT,
            is_rainy_season BOOLEAN
        );
    """,
    "DIM_HEALTH_INITIATIVE": """
        CREATE TABLE IF NOT EXISTS DIM_HEALTH_INITIATIVE (
            initiative_key INT PRIMARY KEY,
            initiative_name VARCHAR(100),
            start_date DATE,
            end_date DATE,
            coverage_percentage DECIMAL(5, 2)
        );
    """,
    "DIM_HEALTHCARE_FACILITY": """
        CREATE TABLE IF NOT EXISTS DIM_HEALTHCARE_FACILITY (
            facility_key INT PRIMARY KEY,
            facility_name VARCHAR(100),
            facility_type VARCHAR(50),
            number_of_beds INT,
            number_of_nurses INT,
            number_of_doctors INT,
            distance_to_community DECIMAL(5, 2),
            facility_location_key INT
        );
    """,
    "DIM_INFRASTRUCTURE": """
        CREATE TABLE IF NOT EXISTS DIM_INFRASTRUCTURE (
            infrastructure_key INT PRIMARY KEY,
            road_quality VARCHAR(50),
            access_to_water BOOLEAN,
            access_to_electricity BOOLEAN,
            mobile_network_coverage BOOLEAN,
            internet_coverage BOOLEAN
        );
    """,
    "DIM_LOCATION": """
        CREATE TABLE IF NOT EXISTS DIM_LOCATION (
            location_key INT PRIMARY KEY,
            district VARCHAR(100),
            region VARCHAR(100),
            population_density DECIMAL(10, 2),
            terrain_type VARCHAR(50),
            distance_to_nearest_health_center DECIMAL(5, 2),
            regional_health_policy VARCHAR(200)
        );
    """,
    "DIM_MEDICAL_HISTORY": """
        CREATE TABLE IF NOT EXISTS DIM_MEDICAL_HISTORY (
            history_key INT PRIMARY KEY,
            previous_complications BOOLEAN,
            chronic_illness BOOLEAN,
            previous_c_section BOOLEAN,
            number_of_pregnancies INT,
            previous_maternal_deaths_in_family BOOLEAN
        );
    """,
    "DIM_PATIENT": """
        CREATE TABLE IF NOT EXISTS DIM_PATIENT (
            patient_key INT PRIMARY KEY,
            age INT,
            education_level VARCHAR(50),
            marital_status VARCHAR(20),
            number_of_children INT,
            antenatal_visits INT,
            complication_history BOOLEAN,
            patient_location_key INT
        );
    """,
    "DIM_PATIENT_OUTCOME": """
        CREATE TABLE IF NOT EXISTS DIM_PATIENT_OUTCOME (
            outcome_key INT PRIMARY KEY,
            delivery_type VARCHAR(50),
            birth_complications BOOLEAN,
            maternal_mortality BOOLEAN,
            neonatal_mortality BOOLEAN,
            recovery_time INT,
            delivery_facility_key INT
        );
    """,
    "DIM_SOCIOECONOMIC": """
        CREATE TABLE IF NOT EXISTS DIM_SOCIOECONOMIC (
            socioeconomic_key INT PRIMARY KEY,
            employment_status VARCHAR(50),
            income_level DECIMAL(10, 2),
            access_to_healthcare BOOLEAN,
            education_level VARCHAR(50),
            insurance_coverage BOOLEAN
        );
    """,
    "DIM_HEALTHWORKER": """
        CREATE TABLE IF NOT EXISTS DIM_HEALTHWORKER (
            worker_key INT PRIMARY KEY,
            worker_id VARCHAR(50),
            worker_role VARCHAR(50),
            years_of_experience INT,
            certification_status BOOLEAN,
            specialization VARCHAR(100),
            healthcare_facility_key INT
        );
    """,
    "FACT_MATERNAL_MORTALITY": """
        CREATE TABLE IF NOT EXISTS FACT_MATERNAL_MORTALITY (
            fact_id INT PRIMARY KEY,
            date_key INT,
            location_key INT,
            patient_key INT,
            healthcare_facility_key INT,
            outcome_key INT,
            socioeconomic_key INT,
            infrastructure_key INT,
            number_of_births INT,
            number_of_maternal_deaths INT
        );
    """
}


### CSV Files Corresponding to the Tables and Data Loading

The code snippet defines a dictionary `csv_files` that maps each table name to its corresponding CSV file storing the data for that table.

A `try` block is used to attempt the following operations:
1. **Table Creation:** The script iterates over the `tables` dictionary containing SQL create table queries and executes them to create dimension and fact tables in the database. It prints a success message for each table created.
2. **Data Loading:** For each table in the `csv_files` dictionary, the script checks if the corresponding CSV file exists. If the file exists, it reads the data from the CSV file using `pandas`, and then loads the data into the respective table in the database using SQLAlchemy's `to_sql` method with `if_exists='append'`. It prints a success message for each table's data loaded.
3. **Exception Handling:** Any exceptions that occur during the process are caught, and an error message is printed.
4. **Finally Block:** A message is printed indicating the completion of the script execution.

This script is responsible for creating tables in the database based on the provided schema and loading data from CSV files into these tables, facilitating the setup of the healthcare data warehouse.

In [10]:
from sqlalchemy import text

# CSV files corresponding to the tables
csv_files = {
    "DIM_DATE": "Data-Mining/data/dataset/DIM_DATE.csv",
    "DIM_HEALTH_INITIATIVE": "Data-Mining/data/DIM_HEALTH_INITIATIVE.csv",
    "DIM_HEALTHCARE_FACILITY": "Data-Mining/data/dataset/DIM_HEALTHCARE_FACILITY.csv",
    "DIM_INFRASTRUCTURE": "Data-Mining/data/dataset/DIM_INFRASTRUCTURE.csv",
    "DIM_LOCATION": "Data-Mining/data/dataset/DIM_LOCATION.csv",
    "DIM_MEDICAL_HISTORY": "Data-Mining/data/dataset/DIM_MEDICAL_HISTORY.csv",
    "DIM_PATIENT": "Data-Mining/data/dataset/DIM_PATIENT.csv",
    "DIM_PATIENT_OUTCOME": "Data-Mining/data/dataset/DIM_PATIENT_OUTCOME.csv",
    "DIM_SOCIOECONOMIC": "Data-Mining/data/dataset/DIM_SOCIOECONOMIC.csv",
    "DIM_HEALTHWORKER": "Data-Mining/data/dataset/DIM_HEALTHWORKER.csv",
    "FACT_MATERNAL_MORTALITY": "Data-Mining/data/dataset/FACT_MATERNAL_MORTALITY.csv"
}

try:
    with engine.connect() as connection:
        # Create tables
        for table_name, create_query in tables.items():
            connection.execute(text(create_query))
            print(f"Table {table_name} created successfully.")

        # Load CSV data into the tables with checking if data already exists
        for table_name, csv_file in csv_files.items():
            if os.path.exists(csv_file):
                data = pd.read_csv(csv_file)
                result = connection.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()
                if result[0] > 0:
                    if_exists_option = 'append'
                else:
                    if_exists_option = 'replace'
                data.to_sql(table_name, engine, if_exists=if_exists_option, index=False)
                print(f"Data loaded into {table_name} from {csv_file} successfully.")
            else:
                print(f"CSV file for {table_name} not found: {csv_file}")

except Exception as e:
    print(f"Error: {e}")
finally:
    print("Script execution completed.")

Table DIM_DATE created successfully.
Table DIM_HEALTH_INITIATIVE created successfully.
Table DIM_HEALTHCARE_FACILITY created successfully.
Table DIM_INFRASTRUCTURE created successfully.
Table DIM_LOCATION created successfully.
Table DIM_MEDICAL_HISTORY created successfully.
Table DIM_PATIENT created successfully.
Table DIM_PATIENT_OUTCOME created successfully.
Table DIM_SOCIOECONOMIC created successfully.
Table DIM_HEALTHWORKER created successfully.
Table FACT_MATERNAL_MORTALITY created successfully.
Error: Not an executable object: 'SELECT COUNT(*) FROM DIM_DATE'
Script execution completed.


### Function to Insert Data into a Table

The code snippet defines a function `insert_data` that inserts data from a CSV file into a specified database table. The function performs the following steps:

1. **Load Dataset:** Reads the data from a CSV file located at the specified `file_path` using `pandas`.
2. **Boolean Column Handling:** If there are boolean columns specified in the `boolean_columns` list, it converts the values ('Yes' and 'No') to boolean values (True and False) in the dataset.
3. **Data Insertion:** Inserts the data into the specified table in the database using the SQLAlchemy connection.
4. **Error Handling:** Catches and prints any exceptions that occur during the data insertion process.

The script also defines a list `table_configs` that contains configurations for inserting data into various tables. Each configuration includes the table name, file path of the CSV containing the data, and a list of boolean columns that need conversion.

A loop iterates through the `table_configs` list, calling the `insert_data` function for each table configuration to insert data into the respective tables in the database.

Upon completion of inserting data into all tables, the script prints "Execution completed."

This function and table insertion process automate the task of populating the dimension and fact tables in the healthcare data warehouse with data from CSV files.

In [5]:


# Function to insert data into a table
def insert_data(table_name, file_path, boolean_columns=None):
    try:
        # Load the dataset
        data = pd.read_csv(file_path)
        
        # Handle boolean column conversions if needed
        if boolean_columns:
            for column in boolean_columns:
                data[column] = data[column].map({'Yes': True, 'No': False})
        
        # Insert data into the database
        with engine.connect() as connection:
            data.to_sql(table_name, connection, if_exists='append', index=False)
        print(f"Data for {table_name} inserted successfully.")
    except Exception as e:
        print(f"Error inserting data for {table_name}: {e}")

# Insert data into all tables
insert_data('DIM_DATE', 'Data-Mining/data/dataset/DIM_DATE.csv', boolean_columns=['is_rainy_season'])
insert_data('DIM_HEALTH_INITIATIVE', 'Data-Mining/data/dataset/DIM_HEALTH_INITIATIVE.csv')
insert_data('DIM_HEALTHCARE_FACILITY', 'Data-Mining/data/dataset/DIM_HEALTHCARE_FACILITY.csv')
insert_data('DIM_INFRASTRUCTURE', 'Data-Mining/data/dataset/DIM_INFRASTRUCTURE.csv', boolean_columns=['access_to_water', 'access_to_electricity', 'mobile_network_coverage', 'internet_coverage'])
insert_data('DIM_LOCATION', 'Data-Mining/data/dataset/DIM_LOCATION.csv')
insert_data('DIM_MEDICAL_HISTORY', 'Data-Mining/data/dataset/DIM_MEDICAL_HISTORY.csv', boolean_columns=['previous_complications', 'chronic_illness', 'previous_c_section', 'previous_maternal_deaths_in_family'])
insert_data('DIM_PATIENT', 'Data-Mining/data/dataset/DIM_PATIENT.csv', boolean_columns=['complication_history'])
insert_data('DIM_PATIENT_OUTCOME', 'Data-Mining/data/dataset/DIM_PATIENT_OUTCOME.csv', boolean_columns=['birth_complications', 'maternal_mortality', 'neonatal_mortality'])
insert_data('DIM_SOCIOECONOMIC', 'Data-Mining/data/dataset/DIM_SOCIOECONOMIC.csv', boolean_columns=['access_to_healthcare', 'insurance_coverage'])
insert_data('DIM_HEALTHWORKER', 'Data-Mining/data/dataset/DIM_HEALTHWORKER.csv', boolean_columns=['certification_status'])
insert_data('FACT_MATERNAL_MORTALITY', 'Data-Mining/data/dataset/FACT_MATERNAL_MORTALITY.csv')

print(" execution completed.")


Data for DIM_DATE inserted successfully.
Data for DIM_HEALTH_INITIATIVE inserted successfully.
Data for DIM_HEALTHCARE_FACILITY inserted successfully.
Data for DIM_INFRASTRUCTURE inserted successfully.
Data for DIM_LOCATION inserted successfully.
Data for DIM_MEDICAL_HISTORY inserted successfully.
Data for DIM_PATIENT inserted successfully.
Data for DIM_PATIENT_OUTCOME inserted successfully.
Data for DIM_SOCIOECONOMIC inserted successfully.
Data for DIM_HEALTHWORKER inserted successfully.
Data for FACT_MATERNAL_MORTALITY inserted successfully.
 execution completed.


### Data Retrieval :

The script iterates through a predefined list of dimension table names, executing SQL queries to retrieve data from each table. Retrieved data is stored in DataFrames, facilitating further processing and analysis. Error handling ensures smooth execution, with a final message indicating completion of the data fetching process.

In [6]:
# List of dimension table names
dimension_tables = [
    "DIM_DATE",
    "DIM_HEALTH_INITIATIVE",
    "DIM_HEALTHCARE_FACILITY",
    "DIM_INFRASTRUCTURE",
    "DIM_LOCATION",
    "DIM_MEDICAL_HISTORY",
    "DIM_PATIENT",
    "DIM_PATIENT_OUTCOME",
    "DIM_SOCIOECONOMIC",
    "DIM_HEALTHWORKER",
    "FACT_MATERNAL_MORTALITY"  # Added FACT table if needed
]

# Dictionary to store DataFrames
dataframes = {}

try:
    with engine.connect() as connection:
        # Fetch each table and store it in the dictionary
        for table_name in dimension_tables:
            query = f"SELECT * FROM {table_name}"
            try:
                # Execute the query and load data into a DataFrame
                df = pd.read_sql(query, connection)
                dataframes[table_name] = df
                print(f"Fetched data from {table_name}. Rows: {len(df)}")
            except Exception as e:
                print(f"Error fetching data from {table_name}: {e}")
except Exception as e:
    print(f"Error establishing database connection: {e}")
finally:
    print("Data fetching process completed.")




Fetched data from DIM_DATE. Rows: 30000
Fetched data from DIM_HEALTH_INITIATIVE. Rows: 30000
Fetched data from DIM_HEALTHCARE_FACILITY. Rows: 30000
Fetched data from DIM_INFRASTRUCTURE. Rows: 30000
Fetched data from DIM_LOCATION. Rows: 30000
Fetched data from DIM_MEDICAL_HISTORY. Rows: 30000
Fetched data from DIM_PATIENT. Rows: 30000
Fetched data from DIM_PATIENT_OUTCOME. Rows: 30000
Fetched data from DIM_SOCIOECONOMIC. Rows: 20000
Fetched data from DIM_HEALTHWORKER. Rows: 30000
Fetched data from FACT_MATERNAL_MORTALITY. Rows: 20000
Data fetching process completed.


### DataFrames for Dimension Tables:

- `dim_date_df`: DataFrame containing data from the "DIM_DATE" table.
- `dim_health_initiative_df`: DataFrame containing data from the "DIM_HEALTH_INITIATIVE" table.
- `dim_healthcare_facility_df`: DataFrame containing data from the "DIM_HEALTHCARE_FACILITY" table.
- `dim_infrastructure_df`: DataFrame containing data from the "DIM_INFRASTRUCTURE" table.
- `dim_location_df`: DataFrame containing data from the "DIM_LOCATION" table.
- `dim_medical_history_df`: DataFrame containing data from the "DIM_MEDICAL_HISTORY" table.
- `dim_patient_df`: DataFrame containing data from the "DIM_PATIENT" table.
- `dim_patient_outcome_df`: DataFrame containing data from the "DIM_PATIENT_OUTCOME" table.
- `dim_socioeconomic_df`: DataFrame containing data from the "DIM_SOCIOECONOMIC" table.
- `dim_healthworker_df`: DataFrame containing data from the "DIM_HEALTHWORKER" table.
- `fact_maternal_mortality_df`: DataFrame containing data from the "FACT_MATERNAL_MORTALITY" table.

These DataFrames hold the retrieved data from the respective dimension tables, allowing for further analysis and manipulation as needed.

In [7]:
dim_date_df = dataframes["DIM_DATE"]
dim_health_initiative_df = dataframes["DIM_HEALTH_INITIATIVE"]
dim_healthcare_facility_df = dataframes["DIM_HEALTHCARE_FACILITY"]
dim_infrastructure_df = dataframes["DIM_INFRASTRUCTURE"]
dim_location_df = dataframes["DIM_LOCATION"]
dim_medical_history_df = dataframes["DIM_MEDICAL_HISTORY"]
dim_patient_df = dataframes["DIM_PATIENT"]
dim_patient_outcome_df = dataframes["DIM_PATIENT_OUTCOME"]
dim_socioeconomic_df = dataframes["DIM_SOCIOECONOMIC"]
dim_healthworker_df = dataframes["DIM_HEALTHWORKER"]
fact_maternal_mortality_df = dataframes["FACT_MATERNAL_MORTALITY"]


In [8]:
dim_health_initiative_df


,initiative_key,initiative_name,start_date,end_date,coverage_percentage
0,1,Initiative 1,2022-10-25,2026-06-05,62.273320
1,2,Initiative 2,2016-02-02,2028-05-17,53.552368
2,3,Initiative 3,2018-09-06,2026-10-21,93.234499
3,4,Initiative 4,2015-12-16,2027-08-30,80.521461
4,5,Initiative 5,2016-11-16,2027-08-14,99.139958
...,...,...,...,...,...
29995,9996,Initiative 9996,2021-10-26,2026-10-21,66.004784
29996,9997,Initiative 9997,2021-03-24,2028-06-05,64.910772
29997,9998,Initiative 9998,2018-07-11,2027-05-26,82.101413
29998,9999,Initiative 9999,2019-06-15,2027-11-05,57.815572


In [9]:
dim_health_initiative_df.describe()

,initiative_key,coverage_percentage
count,30000.000000,30000.000000
mean,5000.500000,75.007439
std,2886.799445,14.364890
min,1.000000,50.000422
25%,2500.750000,62.847824
50%,5000.500000,74.924014
75%,7500.250000,87.480296
max,10000.000000,99.996985
